# Concise Implementation of Linear Regression

We will show you how to implement
the linear regression model
concisely by using high-level APIs

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

Call upon the existing API in a framework to read data

In [5]:
def load_array(data_arrays, batch_size, is_train=True):  
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 1.1686,  0.1964],
         [ 1.1324, -0.3455],
         [ 0.0972,  0.8904],
         [-0.7174, -1.0800],
         [-0.4012, -0.2333],
         [ 0.5376,  0.8296],
         [ 1.6110,  0.5804],
         [ 0.2177,  0.2672],
         [-1.6707,  0.0972],
         [ 1.2844, -0.7029]]),
 tensor([[5.8680],
         [7.6290],
         [1.3579],
         [6.4431],
         [4.2025],
         [2.4808],
         [5.4516],
         [3.7436],
         [0.5257],
         [9.1727]])]

Use a framework's predefined layers

In [6]:
# `nn` is an abbreviation for neural networks
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

Initialize the model parameters

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

The `MSELoss` class computes the mean squared error, also known as squared $L_2$ norm

In [8]:
loss = nn.MSELoss()

Instantiate an `SGD` instance

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

The training loop itself is strikingly similar
to what we did when implementing everything from scratch

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000218
epoch 2, loss 0.000096
epoch 3, loss 0.000097


Compare the model parameters learned by training on finite data
and the actual parameters

In [11]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([-0.0003,  0.0005])
error in estimating b: tensor([0.0008])
